In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os
import s3fs
import pandas as pd

os.chdir("../")

from src.core.configs import settings, fs
from src.dataloader.loader import CustomImageDataset, MaskDataloaders
from src.model.base_model import BaseModel
from src.model.net_model import Net


In [31]:
!pip install pandas
!pip install python-dotenv
!pip install pydantic

In [32]:
train,val= MaskDataloaders(settings.s3_prefix, 0.7).getDataloaders()

In [38]:
cd save_model

/home/onyxia/work/mask_detection/notebooks/save_model


In [39]:
net=Net(2)
model=BaseModel(net)
model.load_model(path='loss_validation0.064.save')
model.fit([train,val])

cuda


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=13456, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

drafts.ipynb  text


torch.device

torch.Size([4, 3, 32, 32])